In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import warnings
warnings.filterwarnings("ignore")
 
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import math
import time
%matplotlib inline

**Знакомимся с LaBSE-en-ru**

In [ ]:
pip install transformers

In [6]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
sentences = ["Hello Word", "Привет Мир", "Raz D", "Раз Д"]
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=64, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)
print(embeddings)

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[-0.0350, -0.0613, -0.0206,  ...,  0.0467, -0.0477, -0.0093],
        [-0.0644, -0.0687, -0.0521,  ...,  0.0139, -0.0362, -0.0213],
        [-0.0526, -0.0535, -0.0092,  ..., -0.0480,  0.0163, -0.0267],
        [-0.0506, -0.0540,  0.0107,  ..., -0.0316, -0.0611, -0.0348]])


Функция LaBSE

In [7]:
def LaBSE_embedding (sentences):    
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=64, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.pooler_output
    embeddings = torch.nn.functional.normalize(embeddings)
    print(embeddings)
    return embeddings

## Читаем данные с файлов.



Данный файл взят из источника: https://github.com/SergeyParamonov/HabraData  
Это один из парсеров Хабра.  
Статьи датируются с 27 сентября по 7 апреля 2014 года.

In [8]:
all_habr = pd.read_csv('/content/drive/My Drive/Colab Notebooks/news-recommender/all.csv')

In [9]:
all_habr.sample(5)

,id,title,author,score,views,favors,isTutorial
8814,post_182072,VPN для обычных пользователей. Реальная необхо...,Maximus43,11,36651,175.0,0
16079,post_206126,Обзор неттопа Foxconn nanoPC AT-7300 на процес...,NeoForce,24,37587,82.0,0
1135,post_156447,Расчет рейтинга доверия в рублях РФ (и других ...,stt,3,4140,9.0,0
17419,post_210426,Удобное переключение wifi в режим точки доступа,conformist,5,9255,82.0,0
5615,post_171411,Дайджест новостей из мира мобильной разработки...,DaryaZ,5,13665,36.0,0


In [10]:
all_habr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19851 entries, 0 to 19850
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           19851 non-null  object 
 1    title       19851 non-null  object 
 2    author      19851 non-null  object 
 3    score       19851 non-null  int64  
 4    views       19851 non-null  int64  
 5    favors      19773 non-null  float64
 6    isTutorial  19851 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 1.1+ MB


In [11]:
all_habr.columns

Index(['id', ' title', ' author', ' score', ' views', ' favors',
       ' isTutorial'],
      dtype='object')

In [12]:
all_habrtittle = all_habr[' title'].tolist()

In [13]:
import random

In [14]:
all_habrsample1k = random.sample(all_habrtittle, 1000)

In [17]:
len(all_habrsample1k)

1000

In [18]:
embeddings_ahs1k = LaBSE_embedding(all_habrsample1k)

tensor([[ 0.0303, -0.0349,  0.0105,  ..., -0.0698, -0.0136, -0.0134],
        [-0.0486, -0.0423, -0.0153,  ..., -0.0314, -0.0331, -0.0319],
        [ 0.0098, -0.0572,  0.0220,  ..., -0.0412, -0.0143,  0.0583],
        ...,
        [-0.0223, -0.0415, -0.0246,  ..., -0.0564, -0.0704, -0.0648],
        [-0.0262, -0.0242, -0.0606,  ...,  0.0099, -0.0514,  0.0019],
        [ 0.0042, -0.0338,  0.0027,  ...,  0.0406, -0.0520, -0.0314]])


In [19]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [20]:
cosine_similarities = cosine_similarity(embeddings_ahs1k)

In [23]:
cosine_similarities

array([[ 1.        ,  0.23182496,  0.2509137 , ...,  0.2567516 ,
         0.03558466,  0.03707391],
       [ 0.23182496,  0.9999999 ,  0.24163592, ...,  0.34509054,
         0.09584635,  0.05486733],
       [ 0.2509137 ,  0.24163592,  1.0000001 , ...,  0.34489265,
         0.07753377, -0.0014026 ],
       ...,
       [ 0.2567516 ,  0.34509054,  0.34489265, ...,  1.0000001 ,
         0.18156129,  0.15572894],
       [ 0.03558466,  0.09584635,  0.07753377, ...,  0.18156129,
         0.9999999 ,  0.33038604],
       [ 0.03707391,  0.05486733, -0.0014026 , ...,  0.15572894,
         0.33038604,  1.        ]], dtype=float32)

In [ ]:
all_habrsample1k

In [31]:
t = np.random.randint(1000)
t

510

In [41]:
similar_indices = cosine_similarities[t].argsort()[:-10:-1]

In [44]:
for i in range(len(similar_indices)):
    print(all_habrsample1k[similar_indices[i]])

Паттерны для новичков: MVC vs MVP vs MVVM
Настройка шаблонов ASP.NET MVC 5 Scaffolding в Visual Studio 2013
Видео: Новые предложения по разработке и тестированию ПО для подписчиков MSDN и другие новости Windows Azure
Новые бесплатные курсы виртуальной академии Microsoft Virtual Academy
Попытка развенчания мифов об OpenVZ comma  или VPS на OpenVZ vs Xen/KVM/Hyper-V/etc
Использование функционала фреймворка MVC4 для авторизации пользователей и использование ролевой модели доступа к сайту
Виджеты сторонних сайтов: «за» и «против»
Понимание Html-кода comma  сгенерированного ненавязчивой валидацией в ASP.Net MVC
AVCONV — конвертация видеофайлов для тега video HTML5 и наложение ватермарка


In [45]:
t = np.random.randint(1000)
t

8

In [46]:
similar_indices = cosine_similarities[t].argsort()[:-10:-1]

In [47]:
for i in range(len(similar_indices)):
    print(all_habrsample1k[similar_indices[i]])

Хакеры получили доступ к базе данных OVH.com
Умельцы получили доступ к ПО Google Chromecast
Статистика ОС на hh.ru
Неизвестный хакер измерил Всемирную паутину
Использование функционала фреймворка MVC4 для авторизации пользователей и использование ролевой модели доступа к сайту
Microsoft начали охоту за ZeroAccess
Покупаем на taobao.com
Поднимаем контроллер домена на Ubuntu Server
Пираты против правообладателей: взгляд инсайдера
